In [1]:
def get_time():
    from datetime import datetime
    from pytz import timezone
    return datetime.now().astimezone(timezone("America/Sao_Paulo")).strftime("%Y_%m_%d_%H_%M_%S")

def get_shape_flow(from_arq_latlon, from_arq_fluxos):
    import os
    import geopandas as gpd
    import pandas as pd
    from shapely.geometry import LineString, Point
    from shapely import wkt

    print("Setando as pastas de destino")
    
    path_shp = "./" + get_time() + "_shp/"
    
    path_geojson = "./" + get_time() + "_geojson/"
    
    path_csv = "./" + get_time() + "_csv/"
    
    path_parquet = "./" + get_time() + "_parquet/"

    print("Criando as pastas de destino (e criando) a pasta /shp")
    os.mkdir(path_parquet)

    print("Lendo o dataset com latitude e longitude")
    df_latlon_ori = pd.read_csv(from_arq_latlon, index_col = False)
    df_latlon_ori[(["NOMESIGMAI", "LATITUDE", "LONGITUDE"])]
    df_latlon_ori["LATITUDE"] = df_latlon_ori["LATITUDE"].astype("str")
    df_latlon_ori["LONGITUDE"] = df_latlon_ori["LONGITUDE"].astype("str")
    df_latlon_ori["origem_latlon"] = "POINT (" + df_latlon_ori["LONGITUDE"] + " " + df_latlon_ori["LATITUDE"] + ")"
    df_latlon_ori.rename({"NOMESIGMAI" : "origem"}, axis = 1, inplace = True)
    
    df_latlon_des = pd.read_csv(from_arq_latlon, index_col = False)
    df_latlon_des["LATITUDE"] = df_latlon_des["LATITUDE"].astype("str")
    df_latlon_des["LONGITUDE"] = df_latlon_des["LONGITUDE"].astype("str")
    df_latlon_des["destino_latlon"] = "POINT (" + df_latlon_des["LONGITUDE"] + " " + df_latlon_des["LATITUDE"] + ")"
    df_latlon_des.drop(["NOMESIG", "NOME", "UF", "GEOCODIGO", "LATITUDEGM", "LONGITUDEG", "RGINT", "RGI", "LATITUDE", "LONGITUDE"], axis = 1, inplace = True)
    df_latlon_des.rename({"NOMESIGMAI" : "destino"}, axis = 1, inplace = True)

    print("Lendo o dataset com os dados de movimentação de passageiros")
    df_left = pd.read_csv(from_arq_fluxos, index_col = False, sep = ";")

    print("Realizando join entre dataset de movimentação + dataset latlon")
    df_left = df_left.merge(df_latlon_ori, on = "origem", how = "left")
    df_left = df_left.merge(df_latlon_des, on = "destino", how = "left")

    df_left["origem_latlon"] = df_left["origem_latlon"].apply(wkt.loads)
    df_left["destino_latlon"] = df_left["destino_latlon"].apply(wkt.loads)

    print("Criando a coluna linestring no dataset de movimentação de passageiros")
    df_left["linestring"] = df_left.apply(lambda row: LineString([row["origem_latlon"], row["destino_latlon"]]), axis = 1)

    print("Criando GeoDataFrame com GeoPandas")
    df_geo = gpd.GeoDataFrame(df_left[["ano", "origem", "destino", "passageiros"]], geometry = df_left["linestring"], crs='epsg:4674')

    df_geo["passageiros"] = gpd.pd.to_numeric(df_geo["passageiros"], downcast = "integer")

    print("Criando coluna id para remoção das linhas duplicadas")
    df_geo["id"] = (df_geo["ano"]).astype("str") + (df_geo["passageiros"] * (df_geo.length * 10)).astype("str").str.replace(".", "", regex = True)

    df_geo["origem_destino"] = df_geo["origem"] + " - " + df_geo["destino"]

    print("Removendo as linhas duplicadas")
    df_geo.drop_duplicates(["id"], keep = "first", inplace = True)

    df_geo.to_parquet(path_parquet + "teste.parquet", compression = "snappy")
    
    print("Tarefa concluída.")
    input()

In [2]:
from path import latlon, fluxos
    
get_shape_flow(latlon, fluxos)

Setando as pastas de destino
Criando as pastas de destino (e criando) a pasta /shp
Lendo o dataset com latitude e longitude
Lendo o dataset com os dados de movimentação de passageiros
Realizando join entre dataset de movimentação + dataset latlon
Criando a coluna linestring no dataset de movimentação de passageiros
Criando GeoDataFrame com GeoPandas
Criando coluna id para remoção das linhas duplicadas


C:\Users\JHZL\AppData\Local\Temp\ipykernel_8196\3443351487.py:60: UserWarning: Geometry is in a geographic CRS. Results from 'length' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  df_geo["id"] = (df_geo["ano"]).astype("str") + (df_geo["passageiros"] * (df_geo.length * 10)).astype("str").str.replace(".", "", regex = True)


Removendo as linhas duplicadas
Tarefa concluída.
